In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from optbinning import OptimalBinning
from sklearn.model_selection import train_test_split
import time


from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

In [4]:
dataset = pd.read_csv('creditcard.csv')
dataset

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [5]:
dataset.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.918649e-15,5.682686e-16,-8.761736e-15,2.811118e-15,-1.552103e-15,2.040130e-15,-1.698953e-15,-1.893285e-16,-3.147640e-15,...,1.473120e-16,8.042109e-16,5.282512e-16,4.456271e-15,1.426896e-15,1.701640e-15,-3.662252e-16,-1.217809e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [6]:
#Excluding TIME AND AMOUNT COLUMNS FOR Features, we will have - 
X = dataset[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28']]
y = dataset['Class']

In [7]:
list_features = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28']

In [8]:
X_list = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28']
y_list = ['target']

In [9]:
pd.set_option('float_format', '{:.16f}'.format)

b = X.describe()
b.T

,count,mean,std,min,25%,50%,75%,max
V1,284807.0000000000000000,0.0000000000000039,1.9586958038574793,-56.4075096313289990,-0.9203733843903220,0.0181087991615309,1.3156416938778650,2.4549299912112099
V2,284807.0000000000000000,0.0000000000000006,1.6513085794769742,-72.7157275629302973,-0.5985499134649160,0.0654855563960555,0.8037238712400945,22.0577289904909009
V3,284807.0000000000000000,-0.0000000000000088,1.5162550051777681,-48.3255893623953980,-0.8903648381551406,0.1798463435635440,1.0271955424655550,9.3825584328211402
V4,284807.0000000000000000,0.0000000000000028,1.4158685749409234,-5.6831711981699504,-0.8486401163312730,-0.0198465294811989,0.7433412894685876,16.8753440335975000
V5,284807.0000000000000000,-0.0000000000000016,1.3802467340314386,-113.7433067111460048,-0.6915970708876575,-0.0543358267364858,0.6119264397351930,34.8016658766686007
V6,284807.0000000000000000,0.0000000000000020,1.3322710897575674,-26.1605059358433003,-0.7682956084604891,-0.2741870765066510,0.3985648963561050,73.3016255459645976
V7,284807.0000000000000000,-0.0000000000000017,1.2370935981826603,-43.5572415712451004,-0.5540758790365226,0.0401030827945351,0.5704360728775986,120.5894939452380044
V8,284807.0000000000000000,-0.0000000000000002,1.1943529026692030,-73.2167184552673973,-0.2086297440394665,0.0223580364366631,0.3273458619234490,20.0072083651213006
V9,284807.0000000000000000,-0.0000000000000031,1.0986320892243226,-13.4340663182300997,-0.6430975702665915,-0.0514287315503490,0.5971390302822686,15.5949946071278003
V10,284807.0000000000000000,0.0000000000000018,1.0888497654025178,-24.5882624372475007,-0.5354257264933235,-0.0929173835961975,0.4539234451395070,23.7451361206545002


In [11]:
dataset['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [13]:
len(dataset['Class'])

284807

## Building Models

### TEST TRAIN SPLIT

In [ ]:
# Split dataset into training and test sets (80/20 split)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split training set into training and validation sets (60/40 split)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

In [ ]:
train = X_train.copy()
train['target'] = y_train

val = X_val.copy()
val['target'] = y_val

test = X_test.copy()
test['target'] = y_test

## Feature Scaling - Min Max Scalar

In [ ]:
scaler = MinMaxScaler()

In [ ]:
train[X_list] = scaler.fit_transform(train[X_list])

In [ ]:
val[X_list] = scaler.transform(val[X_list])
val.head()

In [ ]:
test[X_list] = scaler.transform(test[X_list])
test.head()

In [ ]:
def calculate_metrics(df, x):
    acc = accuracy_score(df['target'], df[x])
    prec = precision_score(df['target'], df[x])
    rec = recall_score(df['target'], df[x])
    f1 = f1_score(df['target'], df[x])
    auc = roc_auc_score(df['target'], df[x])
    
    print(f" Accuracy is {acc}")
    print(f" Precision is {prec:2f}")
    print(f" Recall is {rec:2f}")
    print(f" F1-Score is {f1:2f}")
    print(f" AUC score is {auc:2f}")

## ALGORITHMS

### GridsearchCV for KNN - 
#### 1 - n_neighbors' -  [3,5,10,15,25,50]
#### 2 - 'weights':  'uniform' and  'distance',
#### 3 - Algorithm': brute, kd_tree
#### 4 - Metric -  Euclidean and Manhattan

In [16]:
from sklearn.model_selection import GridSearchCV

In [ ]:
start_time = time.time()



# Define the parameter grid to search over
param_grid = {'n_neighbors': [3, 5, 10, 15, 25, 50],
              'weights': ['uniform', 'distance'],
              #'algorithm': ['brute', 'kd_tree'],
              #'metric': ['euclidean', 'manhattan']
}

# Create a KNN classifier object
knn = KNeighborsClassifier()

# Create a GridSearchCV object and fit it to the data
grid_search = GridSearchCV(knn, param_grid=param_grid)
grid_search.fit(train[X_list], train[y_list])

# Print the best hyperparameters
print("Best hyperparameters: ", grid_search.best_params_)


end_time = time.time()
elapsed_time = end_time - start_time
mm_ss_time = time.strftime("%M:%S", time.gmtime(elapsed_time))

print("Elapsed time:", mm_ss_time)

C:\Users\khann\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\khann\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\khann\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\khann\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

C:\Users\khann\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\khann\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\khann\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\khann\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

C:\Users\khann\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\khann\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\khann\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\khann\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

C:\Users\khann\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\khann\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\khann\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\khann\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC


start_time = time.time()

# Define the hyperparameters and their possible values
param_grid = {
    'C': [0.1, 1000],
    'kernel': ['poly', 'rbf']
    #'decision_function_shape': ['ovr', 'ovo'],
    #'gamma': ['scale', 'auto']
}

# Create the SVM model
svm_model = SVC()

# Perform grid search 
grid_search_svm = GridSearchCV(svm_model, param_grid,verbose = 1)

# Fit the grid search to the data
grid_search_svm.fit(train[X_list], train[y_list])


# Print the best hyperparameters
print("Best hyperparameters: ", grid_search_svm.best_params_)

end_time = time.time()
elapsed_time = end_time - start_time
mm_ss_time = time.strftime("%M:%S", time.gmtime(elapsed_time))

print("Elapsed time:", mm_ss_time)